In [1]:
import os
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal, interpolate
from scipy.interpolate import CubicSpline
import numpy as np
import plotly.graph_objects as go
import pickle
import re
from scipy.fft import fft
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from scipy.signal import welch, hilbert
from scipy.stats import skew, kurtosis, entropy
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import re
import plotly.express as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

participant_id = 1 # Sofia data


In [3]:
def plot_single(all_data, n_samples=5000):
    """Plot single signal with seaborn styling."""
    # Apply seaborn-style settings
    plt.style.use('seaborn')

    col1_name = all_data.columns[0]
    col2_name = all_data.columns[1]

    fig, axes = plt.subplots(2, 1, figsize=(10, 8))

    axes[0].plot(all_data[col1_name].iloc[:n_samples],
                label=f'{col1_name}',
                linewidth=2,
                alpha=0.8)
    axes[0].set_title(f'Signal: {col1_name}', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Samples', fontsize=12)
    axes[0].set_ylabel('Amplitude', fontsize=12)
    axes[0].legend(frameon=True, fancybox=True, shadow=True)
    axes[0].grid(True, linestyle='-', alpha=0.2, color='gray')

    axes[1].plot(all_data[col2_name].iloc[:n_samples],
                label=f'{col2_name}',
                linewidth=2,
                alpha=0.8)
    axes[1].set_title(f'Signal: {col2_name}', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Samples', fontsize=12)
    axes[1].set_ylabel('Amplitude', fontsize=12)
    axes[1].legend(frameon=True, fancybox=True, shadow=True)
    axes[1].grid(True, linestyle='-', alpha=0.2, color='gray')

    plt.tight_layout()
    plt.show()
    return fig

def plot_signals(all_data, vertical_raw_col='vertical_value', vertical_filtered_col='vertical_filtered', vertical_smoothed_col=None,
                 horizontal_raw_col='horizontal_value', horizontal_filtered_col='horizontal_filtered', horizontal_smoothed_col=None, n_samples=5000):
    """Plot signals with seaborn styling."""
    # Apply seaborn-style settings
    plt.style.use('seaborn')

    fig, axes = plt.subplots(2, 1, figsize=(10, 8))

    axes[0].plot(all_data[vertical_raw_col].iloc[:n_samples],
                label='Original Vertical',
                linewidth=2,
                alpha=0.7)
    axes[0].plot(all_data[vertical_filtered_col].iloc[:n_samples],
                label='Filtered Vertical',
                linewidth=2.5,
                alpha=0.9)

    if vertical_smoothed_col is not None:
        axes[0].plot(all_data[vertical_smoothed_col].iloc[:n_samples],
                    label='Smoothed Vertical',
                    linewidth=2.5,
                    alpha=0.9)

    axes[0].set_title('Vertical Signal: Original vs Filtered', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Samples', fontsize=12)
    axes[0].set_ylabel('Amplitude', fontsize=12)
    axes[0].legend(frameon=True, fancybox=True, shadow=True)
    axes[0].grid(True, linestyle='-', alpha=0.2, color='gray')

    axes[1].plot(all_data[horizontal_raw_col].iloc[:n_samples],
                label='Original Horizontal',
                linewidth=2,
                alpha=0.7)
    axes[1].plot(all_data[horizontal_filtered_col].iloc[:n_samples],
                label='Filtered Horizontal',
                linewidth=2.5,
                alpha=0.9)

    if horizontal_smoothed_col is not None:
        axes[1].plot(all_data[horizontal_smoothed_col].iloc[:n_samples],
                    label='Smoothed Horizontal',
                    linewidth=2.5,
                    alpha=0.9)

    axes[1].set_title('Horizontal Signal: Original vs Filtered', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Samples', fontsize=12)
    axes[1].set_ylabel('Amplitude', fontsize=12)
    axes[1].legend(frameon=True, fancybox=True, shadow=True)
    axes[1].grid(True, linestyle='-', alpha=0.2, color='gray')

    plt.tight_layout()
    plt.show()
    return fig

def plot_filter_response(sos, worN=1500, filter_type=None, cutoff_freq=None, bw=None, fs=2.0):
    """Plot filter response with seaborn styling."""
    # Apply seaborn-style settings
    plt.style.use('seaborn')

    w, h = signal.sosfreqz(sos, worN=worN)
    w_normalized = w / np.pi
    freq = w_normalized * (fs/2)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

    db = 20 * np.log10(np.maximum(np.abs(h), 1e-10))
    min_db = np.floor(np.min(db) / 10) * 10
    max_db = np.ceil(np.max(db) / 10) * 10

    if max_db - min_db < 40:
        min_db = max_db - 40

    ax1.plot(freq, db, linewidth=2.5, alpha=0.9)
    ax1.set_ylim([min_db, max_db + 5])

    db_range = max_db - min_db
    if db_range <= 60:
        step = 10
    elif db_range <= 120:
        step = 20
    else:
        step = 40
    db_ticks = np.arange(min_db, max_db + step, step)
    ax1.set_yticks(db_ticks)

    if filter_type and cutoff_freq is not None:
        if filter_type.lower() in ['lowpass', 'highpass']:
            cutoff = cutoff_freq
            ax1.axvline(x=cutoff, color='red', linestyle='--', linewidth=2, alpha=0.8,
                         label=f'Cutoff: {cutoff:.2f} Hz')
            ax1.plot(cutoff, -3, 'o', color='red', markersize=8)
            ax1.annotate('-3 dB', xy=(cutoff, -3), xytext=(cutoff+0.05*(fs/2), -3+5),
                        arrowprops=dict(arrowstyle='->', color='#333333',
                                      connectionstyle='arc3,rad=0.1',
                                      shrink=0.05, lw=1.5),
                        fontsize=11, fontweight='bold', color='#333333')
        elif filter_type.lower() in ['bandpass', 'bandstop']:
            if isinstance(cutoff_freq, (list, tuple)) and len(cutoff_freq) == 2:
                low, high = cutoff_freq
                ax1.axvline(x=low, color='red', linestyle='--', linewidth=2, alpha=0.8,
                             label=f'Lower cutoff: {low:.2f} Hz')
                ax1.axvline(x=high, color='green', linestyle='--', linewidth=2, alpha=0.8,
                             label=f'Upper cutoff: {high:.2f} Hz')
                ax1.plot([low, high], [-3, -3], 'o', color='red', markersize=8)
                ax1.annotate('-3 dB', xy=(low, -3), xytext=(low-0.15*(fs/2), -3+5),
                            arrowprops=dict(arrowstyle='->', color='#333333',
                                          connectionstyle='arc3,rad=0.1',
                                          shrink=0.05, lw=1.5),
                            fontsize=11, fontweight='bold', color='#333333')
                ax1.annotate('-3 dB', xy=(high, -3), xytext=(high+0.05*(fs/2), -3+5),
                            arrowprops=dict(arrowstyle='->', color='#333333',
                                          connectionstyle='arc3,rad=0.1',
                                          shrink=0.05, lw=1.5),
                            fontsize=11, fontweight='bold', color='#333333')
                if filter_type.lower() == 'bandpass':
                    ax1.axvspan(low, high, alpha=0.15, color='green')
                else:
                    ax1.axvspan(0, low, alpha=0.15, color='green')
                    ax1.axvspan(high, fs/2, alpha=0.15, color='green')
            else:
                print("For bandpass/bandstop filters, provide cutoff_freq as [low, high]")

    ax1.grid(True, which='both', linestyle='-', alpha=0.2, color='gray')
    ax1.set_ylabel('Magnitude [dB]', fontsize=12)
    if fs == 2.0:
        ax1.set_xlabel('Normalized frequency (1.0 = Nyquist)', fontsize=12)
    else:
        ax1.set_xlabel('Frequency [Hz]', fontsize=12)

    title = 'Filter Frequency Response'
    if filter_type:
        title = f'{filter_type.capitalize()} Filter Frequency Response'
        if filter_type.lower() in ['lowpass', 'highpass'] and cutoff_freq is not None:
            title += f" (Cutoff: {cutoff_freq:.2f} Hz)"
        elif filter_type.lower() in ['bandpass', 'bandstop'] and isinstance(cutoff_freq, (list, tuple)):
            title += f" (Cutoffs: {cutoff_freq[0]:.2f}-{cutoff_freq[1]:.2f} Hz)"
    ax1.set_title(title, fontsize=14, fontweight='bold')
    if filter_type:
        ax1.legend(loc='best', frameon=True, fancybox=True, shadow=True)

    phase = np.unwrap(np.angle(h))
    ax2.plot(freq, phase, linewidth=2.5, alpha=0.9)
    ax2.grid(True, which='both', linestyle='-', alpha=0.2, color='gray')
    ax2.set_ylabel('Phase [rad]', fontsize=12)
    if fs == 2.0:
        ax2.set_xlabel('Normalized frequency (1.0 = Nyquist)', fontsize=12)
    else:
        ax2.set_xlabel('Frequency [Hz]', fontsize=12)

    phase_ticks = [-np.pi, -0.5*np.pi, 0, 0.5*np.pi, np.pi]
    phase_labels = [r'$-\pi$', r'$-\pi/2$', '0', r'$\pi/2$', r'$\pi$']
    ax2.set_yticks(phase_ticks)
    ax2.set_yticklabels(phase_labels)

    x_limits = [0, fs/2]
    ax1.set_xlim(x_limits)
    ax2.set_xlim(x_limits)

    plt.tight_layout()
    return fig

In [4]:
def plot_signals_interactive(all_data,
                             vertical_raw_col='vertical_value',
                             vertical_filtered_col='vertical_filtered',
                             vertical_smoothed_col=None,
                             horizontal_raw_col='horizontal_value',
                             horizontal_filtered_col='horizontal_filtered',
                             horizontal_smoothed_col=None,
                             n_samples=5000,
                             title='Signal Comparison'):
    """Plot signals with seaborn styling."""
    # Set seaborn style
    sns.set_style("whitegrid")
    sns.set_context("notebook", font_scale=1.1)

    n_samples = min(n_samples, len(all_data))
    data_subset = all_data.iloc[:n_samples]
    x_axis = data_subset.index if isinstance(data_subset.index, pd.DatetimeIndex) else np.arange(n_samples)
    x_title = 'Time' if isinstance(data_subset.index, pd.DatetimeIndex) else 'Samples'

    # Create figure with subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
    fig.suptitle(title, fontsize=16, y=0.98)

    # Plot vertical signals
    ax1.set_title('Vertical Signal Comparison', fontsize=14, pad=10)
    if vertical_raw_col in data_subset.columns:
        ax1.plot(x_axis, data_subset[vertical_raw_col],
                 label='Raw Vertical')

    if vertical_filtered_col in data_subset.columns:
        ax1.plot(x_axis, data_subset[vertical_filtered_col],
                 label='Filtered Vertical')

    if vertical_smoothed_col is not None and vertical_smoothed_col in data_subset.columns:
        ax1.plot(x_axis, data_subset[vertical_smoothed_col],
                 label='Smoothed Vertical')

    ax1.set_ylabel('Amplitude', fontsize=12)
    ax1.legend(loc='upper right', frameon=True, fancybox=True, shadow=True)
    ax1.grid(True)

    # Set x-axis limits to match the data range exactly
    ax1.set_xlim(x_axis[0], x_axis[-1])

    # Plot horizontal signals
    ax2.set_title('Horizontal Signal Comparison', fontsize=14, pad=10)
    if horizontal_raw_col in data_subset.columns:
        ax2.plot(x_axis, data_subset[horizontal_raw_col],
                 label='Raw Horizontal')

    if horizontal_filtered_col in data_subset.columns:
        ax2.plot(x_axis, data_subset[horizontal_filtered_col],
                 label='Filtered Horizontal')

    if horizontal_smoothed_col is not None and horizontal_smoothed_col in data_subset.columns:
        ax2.plot(x_axis, data_subset[horizontal_smoothed_col],
                 label='Smoothed Horizontal')

    ax2.set_xlabel(x_title, fontsize=12)
    ax2.set_ylabel('Amplitude', fontsize=12)
    ax2.legend(loc='upper right', frameon=True, fancybox=True, shadow=True)
    ax2.grid(True)

    # Set x-axis limits to match the data range exactly
    ax2.set_xlim(x_axis[0], x_axis[-1])

    plt.tight_layout()
    plt.close()  # Close the figure to prevent double display
    return fig

def plot_fft_vertical_horizontal(vertical_value, horizontal_value, fs=1000, freq_limit=100):
    """Plot FFT with seaborn styling."""
    # Set seaborn style
    sns.set_style("whitegrid")
    sns.set_context("notebook", font_scale=1.1)

    if isinstance(vertical_value, pd.Series):
        vertical_value = vertical_value.values
    if isinstance(horizontal_value, pd.Series):
        horizontal_value = horizontal_value.values

    vertical_value_centered = vertical_value - np.mean(vertical_value)
    horizontal_value_centered = horizontal_value - np.mean(horizontal_value)

    N = len(vertical_value_centered)
    if N == 0:
        print("Warning: Input data has zero length.")
        return None

    if len(horizontal_value_centered) != N:
        raise ValueError("Vertical and horizontal signals must have the same length.")

    vertical_fft = np.fft.fft(vertical_value_centered)
    horizontal_fft = np.fft.fft(horizontal_value_centered)

    frequencies = np.fft.fftfreq(N, 1/fs)

    positive_freq_mask = frequencies >= 0
    positive_frequencies = frequencies[positive_freq_mask]
    vertical_fft_positive = vertical_fft[positive_freq_mask]
    horizontal_fft_positive = horizontal_fft[positive_freq_mask]

    actual_freq_limit = min(freq_limit, positive_frequencies.max())
    limited_indices = positive_frequencies <= actual_freq_limit

    positive_frequencies_limited = positive_frequencies[limited_indices]
    vertical_magnitude_limited = np.abs(vertical_fft_positive[limited_indices])
    horizontal_magnitude_limited = np.abs(horizontal_fft_positive[limited_indices])

    # Create figure
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    fig.suptitle(f'FFT Analysis (fs={fs} Hz)', fontsize=16, y=0.98)

    # Plot vertical FFT
    ax1.plot(positive_frequencies_limited, vertical_magnitude_limited,
             linewidth=2.5, color='#1f77b4')
    ax1.set_title(f'FFT of Vertical Signal (Centered, 0-{actual_freq_limit:.1f}Hz)',
                  fontsize=14, pad=10)
    ax1.set_xlabel('Frequency (Hz)', fontsize=12)
    ax1.set_ylabel('Magnitude', fontsize=12)
    ax1.set_xlim(0, actual_freq_limit)
    ax1.grid(True, alpha=0.3)

    # Add tight x-axis limits
    if len(positive_frequencies_limited) > 0:
        ax1.set_xlim(positive_frequencies_limited[0], positive_frequencies_limited[-1])

    # Plot horizontal FFT
    ax2.plot(positive_frequencies_limited, horizontal_magnitude_limited,
             linewidth=2.5, color='#ff7f0e')
    ax2.set_title(f'FFT of Horizontal Signal (Centered, 0-{actual_freq_limit:.1f}Hz)',
                  fontsize=14, pad=10)
    ax2.set_xlabel('Frequency (Hz)', fontsize=12)
    ax2.set_ylabel('Magnitude', fontsize=12)
    ax2.set_xlim(0, actual_freq_limit)
    ax2.grid(True, alpha=0.3)

    # Add tight x-axis limits
    if len(positive_frequencies_limited) > 0:
        ax2.set_xlim(positive_frequencies_limited[0], positive_frequencies_limited[-1])

    plt.tight_layout()
    plt.close()  # Close the figure to prevent double display
    return fig

In [5]:
def get_csv_for_participant(participant_id: int, dir):
    files = os.listdir(dir)
    filtered_files = [
        file for file in files
        if file.endswith('.csv') and f'participant_{participant_id}' in file
    ]
    sorted_files = sorted(filtered_files, key=lambda x: int(x.split('_')[1].replace('session', '')))
    return sorted_files

def downsample_signal(signal, original_fs, target_fs):
    num_samples = int(len(signal) * target_fs / original_fs)
    return signal.resample(signal, num_samples)

def bandpass_filter(data, lowcut, highcut, fs, order=2):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    if low <= 0 or high >= 1 or low >= high:
        raise ValueError("Invalid bandpass frequencies.")
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.filtfilt(b, a, data)

def extract_features_for_prediction(eog_horizontal, eog_vertical, window_size=250, step=50):
    features = []

    for start_idx in range(0, len(eog_horizontal) - window_size + 1, step):
        segment_h = eog_horizontal[start_idx:start_idx+window_size]
        segment_v = eog_vertical[start_idx:start_idx+window_size]

        peak_duration_h = np.abs(np.argmax(segment_h) - np.argmin(segment_h))
        peak_duration_v = np.abs(np.argmax(segment_v) - np.argmin(segment_v))
        amplitude_ratio_h = np.max(segment_h) / np.min(segment_h) if np.min(segment_h) != 0 else 0
        amplitude_ratio_v = np.max(segment_v) / np.min(segment_v) if np.min(segment_v) != 0 else 0
        velocity_h = np.max(np.abs(np.gradient(segment_h)))
        velocity_v = np.max(np.abs(np.gradient(segment_v)))
        acceleration_h = np.max(np.abs(np.gradient(np.gradient(segment_h))))
        acceleration_v = np.max(np.abs(np.gradient(np.gradient(segment_v))))
        freq_h = np.fft.fftfreq(len(segment_h))
        psd_h = np.abs(fft(segment_h))**2
        psd_v = np.abs(fft(segment_v))**2
        dominant_freq_h = freq_h[np.argmax(psd_h)]
        dominant_freq_v = freq_h[np.argmax(psd_v)]
        skewness_h = skew(segment_h)
        skewness_v = skew(segment_v)
        kurtosis_h = kurtosis(segment_h)
        kurtosis_v = kurtosis(segment_v)
        entropy_h = -np.sum(np.histogram(segment_h, bins=20, density=True)[0] * np.log(np.histogram(segment_h, bins=20, density=True)[0] + 1e-9))
        entropy_v = -np.sum(np.histogram(segment_v, bins=20, density=True)[0] * np.log(np.histogram(segment_v, bins=20, density=True)[0] + 1e-9))
        peak_to_peak_h = np.ptp(segment_h)
        peak_to_peak_v = np.ptp(segment_v)
        zero_crossings_h = np.count_nonzero(np.diff(np.sign(segment_h)))
        zero_crossings_v = np.count_nonzero(np.diff(np.sign(segment_v)))
        rms_h = np.sqrt(np.mean(np.square(segment_h)))
        rms_v = np.sqrt(np.mean(np.square(segment_v)))
        slope_h = segment_h[-1] - segment_h[0]
        slope_v = segment_v[-1] - segment_v[0]
        derivative_h = np.gradient(segment_h).mean()
        derivative_v = np.gradient(segment_v).mean()

        features.append([
            peak_duration_h, peak_duration_v, amplitude_ratio_h, amplitude_ratio_v,
            velocity_h, velocity_v, acceleration_h, acceleration_v,
            dominant_freq_h, dominant_freq_v, skewness_h, skewness_v,
            kurtosis_h, kurtosis_v, entropy_h, entropy_v,
            peak_to_peak_h, peak_to_peak_v, zero_crossings_h, zero_crossings_v,
            rms_h, rms_v, slope_h, slope_v, derivative_h, derivative_v
        ])

    return np.array(features)


In [6]:
PARTICIPANT = 1
BASE_PATH = "/content/drive/MyDrive/Thesis/Code/Thesis/"

In [7]:
csv_files_for_participant = get_csv_for_participant(participant_id, BASE_PATH)
print(csv_files_for_participant)

['session_1_participant_1_20250327_105914.csv', 'session_2_participant_1_20250327_111452.csv', 'session_3_participant_1_20250327_112708.csv', 'session_4_participant_1_20250327_113821.csv', 'session_5_participant_1_20250327_130157.csv', 'session_6_participant_1_20250327_132415.csv']


In [ ]:
default_fill_value = 2.5

processed_dfs = []
for file in get_csv_for_participant(PARTICIPANT, BASE_PATH):
    df = pd.read_csv(BASE_PATH + file)
    df["corrected_timestamp"] = pd.to_datetime(df["timestamp"] + 3600, unit="s", errors='coerce')
    df.dropna(subset=["corrected_timestamp"], inplace=True)
    df = df.drop_duplicates(subset=["corrected_timestamp"], keep='first')
    df.sort_values("corrected_timestamp", inplace=True)

    if df.empty:
        continue

    # Deduce frequency from the data
    file_start_ts = df["corrected_timestamp"].iloc[0]
    file_end_ts = df["corrected_timestamp"].iloc[-1]
    file_points = len(df)

    # Calculate the time span and deduce frequency
    time_span = file_end_ts - file_start_ts
    if file_points > 1:
        # Calculate frequency based on number of samples and time span
        # Frequency = (number of samples - 1) / time span
        freq_hz = (file_points - 1) / time_span.total_seconds()

        # Convert to appropriate time unit and create frequency string
        if freq_hz >= 1000:
            frequency = f'{int(round(1000/freq_hz))}us'  # microseconds
        elif freq_hz >= 1:
            frequency = f'{int(round(1000/freq_hz))}ms'  # milliseconds
        else:
            frequency = f'{int(round(1/freq_hz))}s'  # seconds
    else:
        # Default to 1ms if only one data point
        frequency = '1ms'

    freq_delta = pd.Timedelta(frequency)

    # Create ideal index based on deduced frequency
    ideal_index = pd.date_range(start=file_start_ts, periods=file_points, freq=frequency)

    df.set_index("corrected_timestamp", inplace=True)

    reindexed_df = df.drop(columns=["timestamp"], errors='ignore').reindex(ideal_index, method='nearest')
    reindexed_df.index.name = "timestamp"

    # Store the frequency with the dataframe for later use
    reindexed_df.attrs['frequency'] = frequency
    reindexed_df.attrs['freq_delta'] = freq_delta

    processed_dfs.append(reindexed_df)

if not processed_dfs:
    raw_data = pd.DataFrame()
    raw_data.index = pd.to_datetime([])
    raw_data.index.name = "timestamp"
else:
    processed_dfs.sort(key=lambda d: d.index.min())

    final_concat_list = [processed_dfs[0]]
    cols = processed_dfs[0].columns

    for i in range(len(processed_dfs) - 1):
        df_prev = processed_dfs[i]
        df_next = processed_dfs[i+1]

        last_timestamp_prev = df_prev.index.max()
        first_timestamp_next = df_next.index.min()

        # Use the frequency from the previous dataframe for gap filling
        freq_delta_prev = df_prev.attrs['freq_delta']
        frequency_prev = df_prev.attrs['frequency']

        gap_start_time = last_timestamp_prev + freq_delta_prev
        gap_end_time = first_timestamp_next - freq_delta_prev

        if gap_start_time <= gap_end_time:
            gap_index = pd.date_range(start=gap_start_time, end=gap_end_time, freq=frequency_prev)
            if not gap_index.empty:
                gap_df = pd.DataFrame(default_fill_value, index=gap_index, columns=cols)
                final_concat_list.append(gap_df)

        final_concat_list.append(df_next)

    raw_data = pd.concat(final_concat_list)

final_concat_list = None
# processed_dfs = None

In [8]:
base_path = "/content/drive/MyDrive/Thesis/Code/Thesis/"
default_fill_value = 2.5
frequency = '1ms'
freq_delta = pd.Timedelta(frequency)

processed_dfs = []
for file in csv_files_for_participant:
    df = pd.read_csv(base_path + file)
    df["corrected_timestamp"] = pd.to_datetime(df["timestamp"] + 3600, unit="s", errors='coerce')
    df.dropna(subset=["corrected_timestamp"], inplace=True)
    df = df.drop_duplicates(subset=["corrected_timestamp"], keep='first')
    df.sort_values("corrected_timestamp", inplace=True)

    if df.empty:
        continue

    file_start_ts = df["corrected_timestamp"].iloc[0]
    file_points = len(df)

    ideal_index = pd.date_range(start=file_start_ts, periods=file_points, freq=frequency)

    df.set_index("corrected_timestamp", inplace=True)

    reindexed_df = df.drop(columns=["timestamp"], errors='ignore').reindex(ideal_index, method='nearest')
    reindexed_df.index.name = "timestamp"

    processed_dfs.append(reindexed_df)

if not processed_dfs:
    raw_data = pd.DataFrame()
    raw_data.index = pd.to_datetime([])
    raw_data.index.name = "timestamp"
else:
    processed_dfs.sort(key=lambda d: d.index.min())

    final_concat_list = [processed_dfs[0]]
    cols = processed_dfs[0].columns

    for i in range(len(processed_dfs) - 1):
        df_prev = processed_dfs[i]
        df_next = processed_dfs[i+1]

        last_timestamp_prev = df_prev.index.max()
        first_timestamp_next = df_next.index.min()

        gap_start_time = last_timestamp_prev + freq_delta
        gap_end_time = first_timestamp_next - freq_delta

        if gap_start_time <= gap_end_time:
            gap_index = pd.date_range(start=gap_start_time, end=gap_end_time, freq=frequency)
            if not gap_index.empty:
                gap_df = pd.DataFrame(default_fill_value, index=gap_index, columns=cols)
                final_concat_list.append(gap_df)

        final_concat_list.append(df_next)

    raw_data = pd.concat(final_concat_list)

final_concat_list = None
# processed_dfs = None

In [ ]:
raw_data.to_parquet(f"data/{PARTICIPANT}_processed_data.parquet")